In [1]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim

import torchdiffeq

from tensorboard_utils import Tensorboard
from tensorboard_utils import tensorboard_event_accumulator

import transformer.Constants as Constants
from transformer.Layers import EncoderLayer, DecoderLayer
from transformer.Modules import ScaledDotProductAttention
from transformer.Models import Decoder, get_attn_key_pad_mask, get_non_pad_mask, get_sinusoid_encoding_table
from transformer.SubLayers import PositionwiseFeedForward

from dataset import TranslationDataset, paired_collate_fn

import model_process
import checkpoints
from node_transformer import NodeTransformer

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook  
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("Torch Version", torch.__version__)

%load_ext autoreload
%autoreload 2



Torch Version 1.1.0


In [ ]:
#!pip install --user --force https://github.com/chengs/tqdm/archive/colab.zip

In [2]:
def prepare_dataloaders(data, batch_size=64, num_workers=2):
    train_loader = torch.utils.data.DataLoader(
        TranslationDataset(
            src_word2idx=data['dict']['src'],
            tgt_word2idx=data['dict']['tgt'],
            src_insts=data['train']['src'],
            tgt_insts=data['train']['tgt']),
        num_workers=num_workers,
        batch_size=batch_size,
        collate_fn=paired_collate_fn,
        shuffle=True)

    valid_loader = torch.utils.data.DataLoader(
        TranslationDataset(
            src_word2idx=data['dict']['src'],
            tgt_word2idx=data['dict']['tgt'],
            src_insts=data['valid']['src'],
            tgt_insts=data['valid']['tgt']),
        num_workers=num_workers,
        batch_size=batch_size,
        collate_fn=paired_collate_fn,
        shuffle=False)
    return train_loader, valid_loader

In [3]:
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda")
print("device", device)

device cuda


In [4]:
data = torch.load("/home/mandubian/datasets/multi30k/multi30k.atok.low.pt")

In [5]:
max_token_seq_len = data['settings'].max_token_seq_len
print(max_token_seq_len)

52


In [6]:
train_loader, val_loader = prepare_dataloaders(data, batch_size=64)

In [ ]:
fst = next(iter(train_loader))
print(fst)
en = ' '.join([train_loader.dataset.src_idx2word[idx] for idx in fst[0][0].numpy()])
ge = ' '.join([train_loader.dataset.tgt_idx2word[idx] for idx in fst[2][0].numpy()])
print(en)
print(ge)

In [ ]:
node_trans.eval()

src, src_idx, tgt, tgt_idx = fst

print("fst", fst)
print("ts", ts)
node_trans(src.to(device), src_idx.to(device), tgt.to(device), tgt_idx.to(device), ts.to(device))

### Create an experiment with a name and a unique ID

In [12]:
exp_name = "node_transformer_multi30k"
unique_id = "2019-06-04_0030"


### Create Model

In [25]:

src_vocab_sz = train_loader.dataset.src_vocab_size
print("src_vocab_sz", src_vocab_sz)
tgt_vocab_sz = train_loader.dataset.tgt_vocab_size
print("tgt_vocab_sz", tgt_vocab_sz)

model = NodeTransformer(
    n_src_vocab=max(src_vocab_sz, tgt_vocab_sz),
    n_tgt_vocab=max(src_vocab_sz, tgt_vocab_sz),
    len_max_seq=max_token_seq_len,
    #emb_src_tgt_weight_sharing=False,
    #d_word_vec=128, d_model=128, d_inner=512,
    n_layers=1, n_head=8, method='dopri5-ext', rtol=1e-3, atol=1e-3)

model = model.to(device)

src_vocab_sz 9795
tgt_vocab_sz 17989


### Create Tensorboard metrics logger

In [24]:
tb = Tensorboard(exp_name, unique_name=unique_id)

Writing TensorBoard events locally to runs/node_transformer_multi30k_2019-06-04_0030


### Create basic optimizer

In [26]:
optimizer = optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.995), eps=1e-9)


### Restore best checkpoint (to restart past training)

In [ ]:
state = checkpoints.restore_best_checkpoint(
    "node_transformer_multi30k", "2019-05-29_1200", "validation", model, optimizer)

print("accuracy", state["acc"])
print("loss", state["loss"])
model = model.to(device)

In [27]:
# Continuous space discretization
timesteps = np.linspace(0., 1, num=6)
timesteps = torch.from_numpy(timesteps).float()

EPOCHS = 50
LOG_INTERVAL = 25

model_process.train(
    exp_name, unique_id,
    model, 
    train_loader, val_loader, timesteps,
    optimizer, device,
    epochs=EPOCHS, tb=tb, log_interval=LOG_INTERVAL,
    #start_epoch=0, best_valid_accu=state["acc"]
)

Loaded model and timesteps to cuda


[ Epoch 0 ]
Adding group train to writers (dict_keys([]))
[Training]  loss: 7.673015976626173, ppl:  2149.55469, accuracy: 15.012 %, elapse: 935993.343ms
Adding group eval to writers (dict_keys(['train']))
[Validation]  loss: 6.588887055954044,  ppl:  726.97134, accuracy: 23.884 %, elapse: 1111.448ms
Checkpointing Validation Model...


[ Epoch 1 ]
[Training]  loss: 6.704420181324197, ppl:  816.00475, accuracy: 28.164 %, elapse: 909428.343ms
[Validation]  loss: 5.997417363321826,  ppl:  402.38823, accuracy: 29.687 %, elapse: 1372.342ms
Checkpointing Validation Model...


[ Epoch 2 ]
[Training]  loss: 6.36515670140328, ppl:  581.23590, accuracy: 31.653 %, elapse: 903455.974ms
[Validation]  loss: 5.68437115573398,  ppl:  294.23276, accuracy: 32.674 %, elapse: 1374.231ms
Checkpointing Validation Model...


[ Epoch 3 ]


...

[ Epoch 46 ]
[Training]  loss: 4.081136880947215, ppl:  59.21275, accuracy: 54.215 %, elapse: 1063959.605ms
[Validation]  loss: 3.308474959031807,  ppl:  27.34339, accuracy: 53.349 %, elapse: 1379.291ms
Checkpointing Validation Model...


[ Epoch 47 ]
[Training]  loss: 4.063845857741888, ppl:  58.19770, accuracy: 54.346 %, elapse: 1065927.727ms
[Validation]  loss: 3.2919823931916685,  ppl:  26.89613, accuracy: 53.428 %, elapse: 1393.012ms
Checkpointing Validation Model...


[ Epoch 48 ]


Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

KeyboardInterrupt: 